# Import Packages

In [ ]:
import arcgis
from arcgis.gis import GIS
import arcgis.features
import datetime
import pandas as pd

# Connect to ArcGIS Online
Credentials should be pulled in from ArcGIS Pro, not stored in unsecured notebook.

In [ ]:
gis = GIS('pro')

In [ ]:
gis.users.me

# Access Layer

In [ ]:
layer = arcgis.gis.Item(gis, itemid="daaad6d492944346976359b1b09653e1")

In [ ]:
layer.url

In [ ]:
layer_flc = arcgis.features.FeatureLayerCollection(layer.url, gis)

# Generate List of Replicas

In [ ]:
replica_list = layer_flc.replicas.get_list()

In [ ]:
replica_list

In [ ]:
print ("There are " + str(len(replica_list)) + " replicas")

# Check out one Replica for testing

In [ ]:
replica1 = layer_flc.replicas.get(replica_list[0]['replicaID'])

In [ ]:
replica1

# Print List of Replicas

In [ ]:
data = []

In [ ]:
for r in replica_list:
    replica = layer_flc.replicas.get(r['replicaID'])
    unixCreationDate = replica["creationDate"]
    unixLastSyncDate = replica["lastSyncDate"]
    creationDate = datetime.datetime.fromtimestamp(unixCreationDate/1000)
    lastSyncDate = datetime.datetime.fromtimestamp(unixLastSyncDate/1000)
    formatted_creation_date = creationDate.strftime('%Y-%m-%d %H:%M:%S')
    formatted_sync_date = lastSyncDate.strftime('%Y-%m-%d %H:%M:%S')
    owner = replica["replicaOwner"]
    replicaID = replica["replicaID"]
    print("Replica: " + replicaID)
    print("Replica Owner: " + owner)
    print("Creation Date: " + formatted_creation_date)
    print("Last Sync Date: " + formatted_sync_date)
    data.append({
        "Replica": replicaID,
        "Replica Owner": owner,
        "Creation Date": formatted_creation_date,
        "Last Sync Date": formatted_sync_date
    })

In [ ]:
df = pd.DataFrame(data)

In [ ]:
df.groupby(['Replica Owner']).size().reset_index(name='Count')

# See How Many Replicas Were Last Synced on Creation Date

In [ ]:
filtered_df = df[df['Last Sync Date'] == df['Creation Date']]

In [ ]:
filtered_df.sort_values(by='Last Sync Date', ascending=False)

In [ ]:
grouped_df = filtered_df.groupby(['Replica Owner']).size().reset_index(name='Count')
grouped_df

# See how many replicas were synced more recently

In [ ]:
filtered_df = df[df['Last Sync Date'] != df['Creation Date']]

In [ ]:
filtered_df.sort_values(by='Last Sync Date', ascending=False)

In [ ]:
grouped_df = filtered_df.groupby(['Replica Owner']).size().reset_index(name='Count')
grouped_df

# Unregister

In [ ]:
for r in replica_list:
    replica = layer_flc.replicas.get(r['replicaID'])
    replica_id = replica["replicaID"]
    print('Unregistering ' + replica_id)
    layer_flc.replicas.unregister(replica_id)